In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import soundfile as sf

from python_speech_features import mfcc

import librosa as lb

from sklearn.model_selection import train_test_split

In [7]:
train = pd.read_csv('urban-sound-classification/train.csv')
test = pd.read_csv('urban-sound-classification/test.csv')

In [8]:
path = []
batch_len = len(train)
for i in range(batch_len):
    path.append('urban-sound-classification/Train/' + str(train.iloc[i,0]) + '.wav')

In [9]:
def time(sample,rate):
    return sample.shape[0]/rate

In [10]:
column = train.Class.unique()

In [11]:
column

array(['siren', 'street_music', 'drilling', 'dog_bark',
       'children_playing', 'gun_shot', 'engine_idling', 'air_conditioner',
       'jackhammer', 'car_horn'], dtype=object)

In [12]:
sample2 = []
rate = []
sample = []
max_length = 0
mfcc_features = []
Y = []
for i in range(len(path)):
    sample2, rate2 = sf.read(path[i])
    if time(sample2,rate2) == 4:
        if sample2.ndim==2:
            sample2 = sample2[:,0]
            sample.append(sample2)
        else:
            sample.append(sample2)
        Y.append(train.iloc[i,1])
        rate.append(rate2)
        mfcc2 = mfcc(sample2,rate2,nfft=2400)
        mfcc_features.append(mfcc2)
    
sample = np.array(sample)
mfcc_features = np.array(mfcc_features)

In [14]:
Y = np.array(Y)

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(mfcc_features, Y, test_size = 0.2, random_state = 4)

In [17]:
X_train2 = np.zeros((len(X_train),400,13))
for i in range(len(X_train)):
    for j in range(len(X_train[i])):
        X_train2[i][j] = X_train[i][j]

In [ ]:
import keras
import tensorflow

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Dropout, Flatten, Activation
from keras import regularizers


model = Sequential()
model.add(Dense(1000, input_dim = 5200, activation = 'relu',kernel_regularizer=regularizers.l2(0.02)))
model.add(Dense(200, activation = 'relu', kernel_regularizer=regularizers.l2(0.02)))
model.add(Dense(10, activation = 'softmax'))

In [ ]:
check = np.zeros((len(Y_train),10))
for i in range(len(Y_train)):
    for j in range(len(column)):
        if column[j]==Y_train[i]:
            check[i][j] = 1

In [ ]:
model = Sequential()
model.add(Conv1D(filters = 1000,kernel_size = 5 ,strides = 3, padding='Same',activation='relu',input_shape=(400,13),kernel_regularizer=regularizers.l2(0.01)))
model.add(Conv1D(filters = 100,kernel_size = 5 ,strides = 3, padding='Same',activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(50))

model.add(Activation('relu'))
model.add(Dense(10,activation='softmax'))
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(X_train2, check,epochs = 30, batch_size = 32)

In [4]:
import pickle

In [ ]:
saved_model = pickle.dump(model,open('CNN_Project.sav','wb'))

In [5]:
sound_model = pickle.load(open('CNN_Project.sav','rb'))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [18]:
X_test2 = np.zeros((len(X_test),400,13))
for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        X_test2[i][j] = X_test[i][j]
        
X_test2.shape

(912, 400, 13)

In [19]:
vector_pred = sound_model.predict(X_test2)

In [20]:
vector_pred[0]

array([5.1053629e-08, 2.1386206e-06, 2.2352029e-05, 7.6371270e-10,
       2.4349009e-09, 6.5583799e-06, 3.5561287e-10, 3.0277033e-07,
       1.5691498e-07, 9.9996841e-01], dtype=float32)

In [22]:
index = np.argmax(vector_pred,axis=1)
Y_pred = []
for i in range(len(index)):
    Y_pred.append(column[index[i]])
Y_pred = np.array(Y_pred)

In [23]:
for i in range(len(Y_pred)):
    print(i+1)
    print(Y_test[i])
    print(Y_pred[i])

1
car_horn
car_horn
2
air_conditioner
air_conditioner
3
children_playing
children_playing
4
drilling
drilling
5
air_conditioner
air_conditioner
6
air_conditioner
air_conditioner
7
jackhammer
jackhammer
8
siren
siren
9
siren
siren
10
dog_bark
siren
11
jackhammer
jackhammer
12
drilling
drilling
13
drilling
drilling
14
dog_bark
dog_bark
15
jackhammer
jackhammer
16
air_conditioner
engine_idling
17
children_playing
children_playing
18
engine_idling
engine_idling
19
siren
siren
20
air_conditioner
engine_idling
21
street_music
street_music
22
jackhammer
jackhammer
23
drilling
drilling
24
siren
siren
25
jackhammer
jackhammer
26
siren
siren
27
engine_idling
children_playing
28
street_music
street_music
29
siren
siren
30
jackhammer
jackhammer
31
jackhammer
jackhammer
32
jackhammer
jackhammer
33
street_music
street_music
34
engine_idling
engine_idling
35
siren
siren
36
engine_idling
engine_idling
37
jackhammer
jackhammer
38
children_playing
children_playing
39
siren
siren
40
dog_bark
dog_bark
41


dog_bark
615
air_conditioner
air_conditioner
616
children_playing
jackhammer
617
siren
siren
618
dog_bark
dog_bark
619
dog_bark
children_playing
620
engine_idling
engine_idling
621
dog_bark
dog_bark
622
dog_bark
children_playing
623
street_music
street_music
624
engine_idling
engine_idling
625
street_music
street_music
626
siren
siren
627
drilling
drilling
628
drilling
drilling
629
street_music
siren
630
street_music
street_music
631
engine_idling
street_music
632
siren
siren
633
jackhammer
jackhammer
634
engine_idling
engine_idling
635
jackhammer
jackhammer
636
drilling
drilling
637
children_playing
street_music
638
dog_bark
children_playing
639
air_conditioner
air_conditioner
640
children_playing
children_playing
641
engine_idling
engine_idling
642
engine_idling
engine_idling
643
siren
siren
644
siren
siren
645
siren
siren
646
air_conditioner
air_conditioner
647
street_music
street_music
648
air_conditioner
street_music
649
car_horn
car_horn
650
air_conditioner
air_conditioner
651
ja

In [24]:
index_test = np.zeros(len(Y_test))
for i in range(len(Y_test)):
    for j in range(len(column)):
        if column[j]==Y_test[i]:
            index_test[i] = j

In [26]:
from sklearn.metrics import accuracy_score

In [27]:
print(accuracy_score(index_test, index))

0.8234649122807017


In [ ]:
path_test = []
for i in range(len(test)):
    path_test.append('urban-sound-classification/Test/' + str(test.iloc[i,0]) + '.wav')

In [ ]:
sample_test = []
rate_test = []
mfcc_features_test = []
index = []
a = 1
for i in range(len(path_test)):
    sample2, rate2 = sf.read(path_test[i])
    rate_test.append(rate2)
    if time(sample2,rate2) == 4:
        if sample2.ndim == 2:
            sample_test.append(sample2[:,0])
            mfcc2 = mfcc(signal=sample2[:,0], samplerate=rate2, nfft=4800)
        else:
            sample_test.append(sample2)
            mfcc2 = mfcc(signal=sample2, samplerate=rate2, nfft=4800)
        index.append(test.iloc[i,0])
        mfcc_features_test.append(mfcc2)
    print(a)
    a+=1

In [ ]:
a = 1
X_test = np.zeros((len(mfcc_features_test),400,13))
for i in range(len(mfcc_features_test)):
    for j in range(len(mfcc_features_test[i])):
        X_test[i][j] = mfcc_features_test[i][j]
    print(a)
    a+=1
X_test.shape

In [ ]:
vector_pred = sound_model.predict(X_test)

In [ ]:
vector_pred[1]

In [ ]:
index_test = np.argmax(vector_pred,axis=1)
Y_pred = []
for i in range(len(index_test)):
    Y_pred.append(column[index_test[i]])
Y_pred = np.array(Y_pred)
index_test

In [ ]:
sound_model.summary()